Notebook for the original dataset loader file

In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.nn.init as init
import torchvision
import argparse
from torch.autograd import Variable
import torch.utils.data as data

from data import v2, v1, AnnotationTransform, VOCDetection, detection_collate, VOCroot, VOC_CLASSES
from data import v2, v1, detection_collate
#from data import MSCOCODetection, COCOAnnotationTransform
from utils.augmentations import SSDAugmentation
from layers.modules import MultiBoxLoss
from ssd import build_ssd
import numpy as np
import time
import sys

from logger import Logger
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
# os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
%set_env CUDA_DEVICE_ORDER = PCI_BUS_ID
%set_env CUDA_VISIBLE_DEVICES = 3

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=3


In [3]:
version = 'v2'
basenet = 'vgg16_reducedfc.pth'
jaccard_threshold = 0.5
batch_size = 32
resume = 'weights/ssd300_0712_bs_78000.pth'
num_workers = 4
iterations = 120000
start_iter = 78001
cuda = True
lr = 1e-3
momentum = 0.9
weight_decay = 5e-4
gamma = 0.1
log_iters = False
visdom = False
send_images_to_visdom = True
save_folder = 'weights/'
voc_root = VOCroot
log_tf = True
log_tf_dir = '.logs/onlyVOC07_baseline_model_after78000/'

In [6]:
dataset = VOCDetection(voc_root, train_sets, None, AnnotationTransform())
dataset1 = VOCDetection(voc_root, train_sets, None, None) 

In [8]:
data = dataset

In [11]:
img, lbl = data[0]

In [13]:
img.size()

torch.Size([3, 375, 500])

In [15]:
lbl

[[0.524, 0.56, 0.646, 0.9013333333333333, 8],
 [0.328, 0.7013333333333334, 0.504, 0.9893333333333333, 8],
 [0.48, 0.5146666666666667, 0.588, 0.7946666666666666, 8]]

In [16]:
bb1 = lbl[0]
print(bb1[0]*375, bb1[1]*500, bb1[2]*375, bb1[3]*500)

196.5 280.0 242.25 450.6666666666667


In [17]:
data1 = dataset1[0]

In [18]:
img, lbl = data1

In [19]:
img.size()

torch.Size([3, 375, 500])

In [22]:
type(lbl)

xml.etree.ElementTree.Element

In [29]:
w,h = 375, 500
for obj in lbl.iter('object'):
    bbox = obj.find('bndbox')
    pts = ['xmin', 'ymin', 'xmax', 'ymax']
    b = []
    bn = []
    for i, pt in enumerate(pts):
        cur_pt = int(bbox.find(pt).text) - 1
        b += [cur_pt]
        cur_pt = cur_pt / h if i % 2 == 0 else cur_pt / h
        bn += [cur_pt]
    print(b)
    print(bn)

[262, 210, 323, 338]
[0.524, 0.42, 0.646, 0.676]
[164, 263, 252, 371]
[0.328, 0.526, 0.504, 0.742]
[4, 243, 66, 373]
[0.008, 0.486, 0.132, 0.746]
[240, 193, 294, 298]
[0.48, 0.386, 0.588, 0.596]
[276, 185, 311, 219]
[0.552, 0.37, 0.622, 0.438]


In [4]:
logger = Logger(log_tf_dir)

In [5]:
if cuda and torch.cuda.is_available():
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
else:
    torch.set_default_tensor_type('torch.FloatTensor')

cfg = (v1, v2)[version == 'v2']

if not os.path.exists(save_folder):
    os.mkdir(save_folder)

train_sets = [('2007', 'trainval')]
# train_sets = 'train'
ssd_dim = 300  # only support 300 now
means = (104, 117, 123)  # only support voc now
num_classes = len(VOC_CLASSES) + 1
batch_size = batch_size
accum_batch_size = 32
iter_size = accum_batch_size / batch_size
max_iter = 120000
weight_decay = 0.0005
stepvalues = (80000, 100000, 120000)
gamma = 0.1
momentum = 0.9

if visdom:
    import visdom
    viz = visdom.Visdom()

ssd_net = build_ssd('train', 300, num_classes)
net = ssd_net

if False:
    net = torch.nn.DataParallel(ssd_net)
    cudnn.benchmark = True

if resume:
    print('Resuming training, loading {}...'.format(resume))
    ssd_net.load_weights(resume)
else:
    vgg_weights = torch.load(save_folder + basenet)
    print('Loading base network...')
    ssd_net.vgg.load_state_dict(vgg_weights)

if cuda and torch.cuda.is_available():
    net = net.cuda()

def xavier(param):
    init.xavier_uniform(param)


def weights_init(m):
    if isinstance(m, nn.Conv2d):
        xavier(m.weight.data)
        m.bias.data.zero_()

if not resume:
    print('Initializing weights...')
    # initialize newly added layers' weights with xavier method
    ssd_net.extras.apply(weights_init)
    ssd_net.loc.apply(weights_init)
    ssd_net.conf.apply(weights_init)

Resuming training, loading weights/ssd300_0712_bs_78000.pth...
Loading weights into state dict...
Finished!


In [6]:
optimizer = optim.SGD(net.parameters(), lr=lr,
                      momentum=momentum, weight_decay=weight_decay)
criterion = MultiBoxLoss(num_classes, 0.5, True, 0, True, 3, 0.5, False, cuda)

In [7]:
def train():
    net.train()
    # loss counters
    loc_loss = 0  # epoch
    conf_loss = 0
    epoch = 0
    print('Loading Dataset...')

    dataset = VOCDetection(voc_root, train_sets, SSDAugmentation(
        ssd_dim, means), AnnotationTransform())

    epoch_size = len(dataset) // batch_size
    print('Training SSD on', dataset.name)
    step_index = 0
    if visdom:
        # initialize visdom loss plot
        lot = viz.line(
            X=torch.zeros((1,)).cpu(),
            Y=torch.zeros((1, 3)).cpu(),
            opts=dict(
                xlabel='Iteration',
                ylabel='Loss',
                title='Current SSD Training Loss',
                legend=['Loc Loss', 'Conf Loss', 'Loss']
            )
        )
        epoch_lot = viz.line(
            X=torch.zeros((1,)).cpu(),
            Y=torch.zeros((1, 3)).cpu(),
            opts=dict(
                xlabel='Epoch',
                ylabel='Loss',
                title='Epoch SSD Training Loss',
                legend=['Loc Loss', 'Conf Loss', 'Loss']
            )
        )
    batch_iterator = None
    data_loader = data.DataLoader(dataset, batch_size, num_workers=num_workers,
                                  shuffle=True, collate_fn=detection_collate, pin_memory=True)
    for iteration in range(start_iter, max_iter):
        if (not batch_iterator) or (iteration % epoch_size == 0):
            # create batch iterator
            batch_iterator = iter(data_loader)
        if iteration in stepvalues:
            step_index += 1
            adjust_learning_rate(optimizer, gamma, step_index)
            if visdom:
                viz.line(
                    X=torch.ones((1, 3)).cpu() * epoch,
                    Y=torch.Tensor([loc_loss, conf_loss,
                        loc_loss + conf_loss]).unsqueeze(0).cpu() / epoch_size,
                    win=epoch_lot,
                    update='append'
                )
            # reset epoch loss counters
            loc_loss = 0
            conf_loss = 0
            epoch += 1

        # load train data
        images, targets = next(batch_iterator)

        if cuda:
            images = Variable(images.cuda())
            targets = [Variable(anno.cuda(), volatile=True) for anno in targets]
        else:
            images = Variable(images)
            targets = [Variable(anno, volatile=True) for anno in targets]
        # forward
        t0 = time.time()
        out = net(images)
        # backprop
        optimizer.zero_grad()
        loss_l, loss_c = criterion(out, targets)
        loss = loss_l + loss_c
        loss.backward()
        optimizer.step()
        t1 = time.time()
        loc_loss += loss_l.data[0]
        conf_loss += loss_c.data[0]
        if iteration % 10 == 0:
            print('Timer: %.4f sec.' % (t1 - t0))
            print('iter ' + repr(iteration) + ' || Loss: %.4f ||' % (loss.data[0]), end=' ')
            if visdom and send_images_to_visdom:
                random_batch_index = np.random.randint(images.size(0))
                viz.image(images.data[random_batch_index].cpu().numpy())
            if log_tf:
                info = {
                    'conf_loss' : loss_c.data[0],
                    'loc_loss' : loss_l.data[0],
                    'loss' : loss.data[0]
                }
                for tag, val in info.items():
                    logger.scalar_summary(tag, val, iteration)
                
                def to_np(x):
                    return x.data.cpu().numpy()
                
                for tag, val in net.named_parameters():
                    tag = tag.replace('.', '/')
                    logger.histo_summary(tag, to_np(val), iteration)
                    logger.histo_summary(tag+'/grad', to_np(val.grad), iteration)
        if visdom:
            viz.line(
                X=torch.ones((1, 3)).cpu() * iteration,
                Y=torch.Tensor([loss_l.data[0], loss_c.data[0],
                    loss_l.data[0] + loss_c.data[0]]).unsqueeze(0).cpu(),
                win=lot,
                update='append'
            )
            # hacky fencepost solution for 0th epoch plot
            if iteration == 0:
                viz.line(
                    X=torch.zeros((1, 3)).cpu(),
                    Y=torch.Tensor([loc_loss, conf_loss,
                        loc_loss + conf_loss]).unsqueeze(0).cpu(),
                    win=epoch_lot,
                    update=True
                )
        if iteration % 2000 == 0:
            print('Saving state, iter:', iteration)
            torch.save(ssd_net.state_dict(), save_folder + 'ssd300_0712_bs_' +
                       repr(iteration) + '.pth')
    torch.save(ssd_net.state_dict(), save_folder + '' + version + '.pth')

def adjust_learning_rate(optimizer, gamma, step):
    """Sets the learning rate to the initial LR decayed by 10 at every specified step
    # Adapted from PyTorch Imagenet example:
    # https://github.com/pytorch/examples/blob/master/imagenet/main.py
    """
    global lr
    lr = lr * (gamma ** (step))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


In [8]:
train()

Loading Dataset...
Training SSD on VOC0712


/users/gpu/utkrsh/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/autograd/_functions/tensor.py:450: UserWarning: mask is not broadcastable to self, but they have the same number of elements.  Falling back to deprecated pointwise behavior.
  return tensor.masked_fill_(mask, value)


Timer: 0.4236 sec.
iter 78010 || Loss: 1.8973 || Timer: 0.4222 sec.
iter 78020 || Loss: 2.2902 || Timer: 0.4287 sec.
iter 78030 || Loss: 2.5510 || Timer: 0.4229 sec.
iter 78040 || Loss: 2.2389 || Timer: 0.4358 sec.
iter 78050 || Loss: 2.2041 || Timer: 0.4425 sec.
iter 78060 || Loss: 2.6034 || Timer: 0.4424 sec.
iter 78070 || Loss: 1.7458 || Timer: 0.4576 sec.
iter 78080 || Loss: 2.3060 || Timer: 0.4485 sec.
iter 78090 || Loss: 2.5675 || Timer: 0.4424 sec.
iter 78100 || Loss: 1.9981 || Timer: 0.4468 sec.
iter 78110 || Loss: 2.2646 || Timer: 0.4528 sec.
iter 78120 || Loss: 2.1343 || Timer: 0.4409 sec.
iter 78130 || Loss: 2.3894 || Timer: 0.4456 sec.
iter 78140 || Loss: 2.6417 || Timer: 0.4471 sec.
iter 78150 || Loss: 2.8772 || Timer: 0.4343 sec.
iter 78160 || Loss: 1.8000 || Timer: 0.4530 sec.
iter 78170 || Loss: 2.2254 || Timer: 0.4515 sec.
iter 78180 || Loss: 2.5451 || Timer: 0.4476 sec.
iter 78190 || Loss: 2.5224 || Timer: 0.4484 sec.
iter 78200 || Loss: 2.5500 || Timer: 0.4509 sec.
i

iter 79680 || Loss: 2.1052 || Timer: 0.4411 sec.
iter 79690 || Loss: 2.1169 || Timer: 0.4422 sec.
iter 79700 || Loss: 2.1595 || Timer: 0.4436 sec.
iter 79710 || Loss: 2.9799 || Timer: 0.4284 sec.
iter 79720 || Loss: 1.9990 || Timer: 0.4476 sec.
iter 79730 || Loss: 2.0559 || Timer: 0.4520 sec.
iter 79740 || Loss: 2.4498 || Timer: 0.4509 sec.
iter 79750 || Loss: 2.0791 || Timer: 0.4453 sec.
iter 79760 || Loss: 2.2977 || Timer: 0.4481 sec.
iter 79770 || Loss: 2.2894 || Timer: 0.4423 sec.
iter 79780 || Loss: 2.3893 || Timer: 0.4471 sec.
iter 79790 || Loss: 2.3340 || Timer: 0.4476 sec.
iter 79800 || Loss: 1.8531 || Timer: 0.4432 sec.
iter 79810 || Loss: 2.3765 || Timer: 0.4486 sec.
iter 79820 || Loss: 2.2751 || Timer: 0.4447 sec.
iter 79830 || Loss: 2.2120 || Timer: 0.4507 sec.
iter 79840 || Loss: 2.6048 || Timer: 0.4491 sec.
iter 79850 || Loss: 2.6569 || Timer: 0.4449 sec.
iter 79860 || Loss: 2.0180 || Timer: 0.4469 sec.
iter 79870 || Loss: 2.0758 || Timer: 0.4329 sec.
iter 79880 || Loss: 

iter 81350 || Loss: 1.6652 || Timer: 0.4423 sec.
iter 81360 || Loss: 1.8857 || Timer: 0.4459 sec.
iter 81370 || Loss: 1.8622 || Timer: 0.4467 sec.
iter 81380 || Loss: 2.0220 || Timer: 0.4464 sec.
iter 81390 || Loss: 1.8035 || Timer: 0.4510 sec.
iter 81400 || Loss: 1.5414 || Timer: 0.4424 sec.
iter 81410 || Loss: 1.7213 || Timer: 0.4460 sec.
iter 81420 || Loss: 2.1691 || Timer: 0.4416 sec.
iter 81430 || Loss: 1.8116 || Timer: 0.4328 sec.
iter 81440 || Loss: 2.2164 || Timer: 0.4483 sec.
iter 81450 || Loss: 2.0362 || Timer: 0.4514 sec.
iter 81460 || Loss: 1.8735 || Timer: 0.4493 sec.
iter 81470 || Loss: 2.3187 || Timer: 0.4480 sec.
iter 81480 || Loss: 1.5546 || Timer: 0.4451 sec.
iter 81490 || Loss: 1.6952 || Timer: 0.4557 sec.
iter 81500 || Loss: 1.9963 || Timer: 0.4468 sec.
iter 81510 || Loss: 1.8705 || Timer: 0.4570 sec.
iter 81520 || Loss: 2.0234 || Timer: 0.4500 sec.
iter 81530 || Loss: 1.8976 || Timer: 0.4455 sec.
iter 81540 || Loss: 1.8061 || Timer: 0.4474 sec.
iter 81550 || Loss: 

...

iter 104620 || Loss: 2.0747 || Timer: 0.4483 sec.
iter 104630 || Loss: 1.5917 || Timer: 0.4541 sec.
iter 104640 || Loss: 1.5357 || Timer: 0.4526 sec.
iter 104650 || Loss: 1.7685 || Timer: 0.4542 sec.
iter 104660 || Loss: 1.5666 || Timer: 0.4487 sec.
iter 104670 || Loss: 1.9397 || Timer: 0.4305 sec.
iter 104680 || Loss: 1.8163 || Timer: 0.4502 sec.
iter 104690 || Loss: 1.6735 || Timer: 0.4483 sec.
iter 104700 || Loss: 1.5700 || Timer: 0.4523 sec.
iter 104710 || Loss: 1.5818 || Timer: 0.4500 sec.
iter 104720 || Loss: 1.3908 || Timer: 0.4462 sec.
iter 104730 || Loss: 1.4556 || Timer: 0.4471 sec.
iter 104740 || Loss: 1.8030 || Timer: 0.4544 sec.
iter 104750 || Loss: 1.7207 || Timer: 0.4515 sec.
iter 104760 || Loss: 1.8112 || Timer: 0.4506 sec.
iter 104770 || Loss: 2.1995 || Timer: 0.4518 sec.
iter 104780 || Loss: 2.1518 || Timer: 0.4484 sec.
iter 104790 || Loss: 1.6402 || Timer: 0.4494 sec.
iter 104800 || Loss: 1.4394 || Timer: 0.4544 sec.
iter 104810 || Loss: 1.8448 || Timer: 0.4550 sec.


iter 107890 || Loss: 1.6214 || Timer: 0.4477 sec.
iter 107900 || Loss: 1.4545 || Timer: 0.4521 sec.
iter 107910 || Loss: 1.7690 || Timer: 0.4527 sec.
iter 107920 || Loss: 1.7347 || Timer: 0.4494 sec.
iter 107930 || Loss: 1.7290 || Timer: 0.4501 sec.
iter 107940 || Loss: 1.8055 || Timer: 0.4534 sec.
iter 107950 || Loss: 1.6304 || Timer: 0.4544 sec.
iter 107960 || Loss: 1.8173 || Timer: 0.4540 sec.
iter 107970 || Loss: 1.6921 || Timer: 0.4483 sec.
iter 107980 || Loss: 1.4631 || Timer: 0.4513 sec.
iter 107990 || Loss: 1.6876 || Timer: 0.4488 sec.
iter 108000 || Loss: 1.6227 || Saving state, iter: 108000
Timer: 0.4280 sec.
iter 108010 || Loss: 1.6048 || Timer: 0.4521 sec.
iter 108020 || Loss: 1.5294 || Timer: 0.4524 sec.
iter 108030 || Loss: 1.6162 || Timer: 0.4518 sec.
iter 108040 || Loss: 1.9416 || Timer: 0.4492 sec.
iter 108050 || Loss: 1.6406 || Timer: 0.4542 sec.
iter 108060 || Loss: 1.4569 || Timer: 0.4491 sec.
iter 108070 || Loss: 1.6069 || Timer: 0.4498 sec.
iter 108080 || Loss: 1.

iter 111150 || Loss: 1.8966 || Timer: 0.4509 sec.
iter 111160 || Loss: 1.7033 || Timer: 0.4542 sec.
iter 111170 || Loss: 1.9136 || Timer: 0.4490 sec.
iter 111180 || Loss: 1.6352 || Timer: 0.4507 sec.
iter 111190 || Loss: 1.8246 || Timer: 0.4526 sec.
iter 111200 || Loss: 1.6676 || Timer: 0.4485 sec.
iter 111210 || Loss: 1.6983 || Timer: 0.4580 sec.
iter 111220 || Loss: 1.8682 || Timer: 0.4288 sec.
iter 111230 || Loss: 1.7935 || Timer: 0.4508 sec.
iter 111240 || Loss: 1.8636 || Timer: 0.4482 sec.
iter 111250 || Loss: 1.4701 || Timer: 0.4470 sec.
iter 111260 || Loss: 1.6232 || Timer: 0.4505 sec.
iter 111270 || Loss: 2.2079 || Timer: 0.4578 sec.
iter 111280 || Loss: 2.1891 || Timer: 0.4583 sec.
iter 111290 || Loss: 1.3903 || Timer: 0.4524 sec.
iter 111300 || Loss: 1.7315 || Timer: 0.4493 sec.
iter 111310 || Loss: 1.6248 || Timer: 0.4455 sec.
iter 111320 || Loss: 1.6490 || Timer: 0.4506 sec.
iter 111330 || Loss: 1.2696 || Timer: 0.4498 sec.
iter 111340 || Loss: 2.1812 || Timer: 0.4489 sec.


iter 114410 || Loss: 1.5759 || Timer: 0.4535 sec.
iter 114420 || Loss: 2.0876 || Timer: 0.4526 sec.
iter 114430 || Loss: 1.7180 || Timer: 0.4528 sec.
iter 114440 || Loss: 1.9113 || Timer: 0.4517 sec.
iter 114450 || Loss: 1.7864 || Timer: 0.4493 sec.
iter 114460 || Loss: 1.6855 || Timer: 0.4533 sec.
iter 114470 || Loss: 1.7190 || Timer: 0.4537 sec.
iter 114480 || Loss: 1.6158 || Timer: 0.4460 sec.
iter 114490 || Loss: 1.1823 || Timer: 0.4527 sec.
iter 114500 || Loss: 1.5293 || Timer: 0.4638 sec.
iter 114510 || Loss: 1.5597 || Timer: 0.4816 sec.
iter 114520 || Loss: 1.7260 || Timer: 0.4496 sec.
iter 114530 || Loss: 1.4274 || Timer: 0.4533 sec.
iter 114540 || Loss: 1.5168 || Timer: 0.4521 sec.
iter 114550 || Loss: 2.0693 || Timer: 0.4536 sec.
iter 114560 || Loss: 2.3460 || Timer: 0.4495 sec.
iter 114570 || Loss: 1.5749 || Timer: 0.4512 sec.
iter 114580 || Loss: 1.2384 || Timer: 0.4537 sec.
iter 114590 || Loss: 1.6538 || Timer: 0.4498 sec.
iter 114600 || Loss: 1.6292 || Timer: 0.4542 sec.


iter 117680 || Loss: 1.6481 || Timer: 0.4489 sec.
iter 117690 || Loss: 1.7726 || Timer: 0.4495 sec.
iter 117700 || Loss: 1.9008 || Timer: 0.4537 sec.
iter 117710 || Loss: 1.4093 || Timer: 0.4463 sec.
iter 117720 || Loss: 1.5594 || Timer: 0.4503 sec.
iter 117730 || Loss: 1.3703 || Timer: 0.4498 sec.
iter 117740 || Loss: 1.2638 || Timer: 0.4517 sec.
iter 117750 || Loss: 2.1433 || Timer: 0.4523 sec.
iter 117760 || Loss: 2.1270 || Timer: 0.4506 sec.
iter 117770 || Loss: 1.9936 || Timer: 0.4787 sec.
iter 117780 || Loss: 1.4584 || Timer: 0.4521 sec.
iter 117790 || Loss: 1.8727 || Timer: 0.4499 sec.
iter 117800 || Loss: 1.3562 || Timer: 0.4522 sec.
iter 117810 || Loss: 1.9089 || Timer: 0.4481 sec.
iter 117820 || Loss: 1.6676 || Timer: 0.4533 sec.
iter 117830 || Loss: 1.6567 || Timer: 0.4468 sec.
iter 117840 || Loss: 1.6246 || Timer: 0.4469 sec.
iter 117850 || Loss: 1.7072 || Timer: 0.4436 sec.
iter 117860 || Loss: 1.7672 || Timer: 0.4599 sec.
iter 117870 || Loss: 1.8820 || Timer: 0.4514 sec.
